In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/travel-behavior-insights/sample_submission.csv
/kaggle/input/travel-behavior-insights/train.csv
/kaggle/input/travel-behavior-insights/test.csv


In [2]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
train_df = pd.read_csv("/kaggle/input/travel-behavior-insights/train.csv")
test_df = pd.read_csv("/kaggle/input/travel-behavior-insights/test.csv")
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12654 entries, 0 to 12653
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   trip_id                      12654 non-null  object 
 1   country                      12424 non-null  object 
 2   age_group                    12646 non-null  object 
 3   travel_companions            11917 non-null  object 
 4   num_females                  12652 non-null  float64
 5   num_males                    12650 non-null  float64
 6   main_activity                12526 non-null  object 
 7   visit_purpose                12654 non-null  object 
 8   is_first_visit               12555 non-null  object 
 9   mainland_stay_nights         12654 non-null  int64  
 10  island_stay_nights           12654 non-null  int64  
 11  tour_type                    12654 non-null  object 
 12  intl_transport_included      12507 non-null  object 
 13  info_source     

In [3]:
'''It can be seen that the target variable spend_category has some rows where its value is null. It is best to drop these 
rows (as there are not many compared to the overall dataset size) rather than imputing values and introducing bias'''

train_df.dropna(subset=['spend_category'], inplace=True)

#plotting missing percentages after dropping the rows with missing spend_category
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12620 entries, 0 to 12653
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   trip_id                      12620 non-null  object 
 1   country                      12393 non-null  object 
 2   age_group                    12612 non-null  object 
 3   travel_companions            11887 non-null  object 
 4   num_females                  12618 non-null  float64
 5   num_males                    12616 non-null  float64
 6   main_activity                12492 non-null  object 
 7   visit_purpose                12620 non-null  object 
 8   is_first_visit               12522 non-null  object 
 9   mainland_stay_nights         12620 non-null  int64  
 10  island_stay_nights           12620 non-null  int64  
 11  tour_type                    12620 non-null  object 
 12  intl_transport_included      12474 non-null  object 
 13  info_source          

In [4]:
#For the numerical cols num_males and num_females which have very low missing percentages, we impute the missing values with the median
train_df['has_special_requirements_missing'] = train_df['has_special_requirements'].isnull().astype(int)
test_df['has_special_requirements_missing'] = test_df['has_special_requirements'].isnull().astype(int)
train_df['arrival_weather_missing'] = train_df['arrival_weather'].isnull().astype(int)
test_df['arrival_weather_missing'] = test_df['arrival_weather'].isnull().astype(int)

missing_numerical_cols = ['num_females', 'num_males']
for col in missing_numerical_cols:
    median_val = train_df[col].median()
    train_df[col] = train_df[col].fillna(median_val)
    test_df[col] = test_df[col].fillna(median_val)

#For categorical columns with low missing percentages, we impute the missing values with the mode
missing_categorical_cols = ['country','age_group','main_activity','travel_companions', 'is_first_visit', 'intl_transport_included', 'accomodation_included', 'food_included', 'insurance_included', 'days_booked_before_trip', 'total_trip_days']
for col in missing_categorical_cols:
    mode_val = train_df[col].mode()[0]
    train_df[col] = train_df[col].fillna(mode_val)
    test_df[col] = test_df[col].fillna(mode_val)

#Fill with "none" - missingness indicator already created
train_df['has_special_requirements'] = train_df['has_special_requirements'].fillna("none")
test_df['has_special_requirements'] = test_df['has_special_requirements'].fillna("none")

#For arrival_weather we fill the missing values with "other"
train_df['arrival_weather'] = train_df['arrival_weather'].fillna("none")
test_df['arrival_weather'] = test_df['arrival_weather'].fillna("none")

#Feature Engineering
# Total travelers
train_df['total_travelers'] = train_df['num_females'] + train_df['num_males']
test_df['total_travelers'] = test_df['num_females'] + test_df['num_males']

# Gender ratio
train_df['female_ratio'] = train_df['num_females'] / (train_df['total_travelers'] + 1e-6)
test_df['female_ratio'] = test_df['num_females'] / (test_df['total_travelers'] + 1e-6)

# Stay nights features
train_df['total_stay_nights'] = train_df['mainland_stay_nights'] + train_df['island_stay_nights']
test_df['total_stay_nights'] = test_df['mainland_stay_nights'] + test_df['island_stay_nights']

train_df['island_ratio'] = train_df['island_stay_nights'] / (train_df['total_stay_nights'] + 1e-6)
test_df['island_ratio'] = test_df['island_stay_nights'] / (test_df['total_stay_nights'] + 1e-6)

# Mainland preference (binary)
train_df['prefers_mainland'] = (train_df['mainland_stay_nights'] > train_df['island_stay_nights']).astype(int)
test_df['prefers_mainland'] = (test_df['mainland_stay_nights'] > test_df['island_stay_nights']).astype(int)


included_cols = ['intl_transport_included', 'accomodation_included', 'food_included',
                 'domestic_transport_included', 'sightseeing_included', 
                 'guide_included', 'insurance_included']

# Fill missing with mode for each
for col in included_cols:
    if train_df[col].isnull().sum() > 0:
        mode_val = train_df[col].mode()[0]
        train_df[col] = train_df[col].fillna(mode_val)
        test_df[col] = test_df[col].fillna(mode_val)

# Count services
train_df['num_services_included'] = train_df[included_cols].apply(
    lambda x: (x == 'Yes').sum(), axis=1
)
test_df['num_services_included'] = test_df[included_cols].apply(
    lambda x: (x == 'Yes').sum(), axis=1
)

# Service ratio
train_df['service_ratio'] = train_df['num_services_included'] / len(included_cols)
test_df['service_ratio'] = test_df['num_services_included'] / len(included_cols)

# Is comprehensive package (most services included)
train_df['is_comprehensive_package'] = (train_df['num_services_included'] >= 5).astype(int)
test_df['is_comprehensive_package'] = (test_df['num_services_included'] >= 5).astype(int)

train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12620 entries, 0 to 12653
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   trip_id                           12620 non-null  object 
 1   country                           12620 non-null  object 
 2   age_group                         12620 non-null  object 
 3   travel_companions                 12620 non-null  object 
 4   num_females                       12620 non-null  float64
 5   num_males                         12620 non-null  float64
 6   main_activity                     12620 non-null  object 
 7   visit_purpose                     12620 non-null  object 
 8   is_first_visit                    12620 non-null  object 
 9   mainland_stay_nights              12620 non-null  int64  
 10  island_stay_nights                12620 non-null  int64  
 11  tour_type                         12620 non-null  object 
 12  intl_tran

In [5]:
pip install pycountry_convert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sklearn.model_selection import KFold
from pycountry_convert import country_name_to_country_alpha2, country_alpha2_to_continent_code


#Encoding

#country encoding
from pycountry_convert import country_name_to_country_alpha2, country_alpha2_to_continent_code

def get_continent(country_name):
    try:
        alpha2 = country_name_to_country_alpha2(country_name)
        continent = country_alpha2_to_continent_code(alpha2)
        return continent
    except:
        return 'Unknown'

# Fill missing countries before encoding
train_df['country'] = train_df['country'].fillna('Unknown')
test_df['country'] = test_df['country'].fillna('Unknown')

# A. Continent
train_df['continent'] = train_df['country'].apply(get_continent)
test_df['continent'] = test_df['country'].apply(get_continent)

# B. Country frequency
country_freq = train_df['country'].value_counts()
train_df['country_freq'] = train_df['country'].map(country_freq).fillna(0)
test_df['country_freq'] = test_df['country'].map(country_freq).fillna(0)

# C. Top countries
top_countries = train_df['country'].value_counts().head(15).index.tolist()
train_df['is_top_country'] = train_df['country'].isin(top_countries).astype(int)
test_df['is_top_country'] = test_df['country'].isin(top_countries).astype(int)

# D. Target encoding for country
def target_encode_country(train_df, test_df, col='country', target='spend_category', n_splits=5):
    train_df[f'{col}_target_enc'] = 0.0
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for train_idx, val_idx in kf.split(train_df, train_df[target]):
        means = train_df.iloc[train_idx].groupby(col)[target].mean()
        train_df.loc[train_df.index[val_idx], f'{col}_target_enc'] = \
            train_df.iloc[val_idx][col].map(means)
    
    global_mean = train_df[target].mean()
    train_df[f'{col}_target_enc'] = train_df[f'{col}_target_enc'].fillna(global_mean)
    
    means = train_df.groupby(col)[target].mean()
    test_df[f'{col}_target_enc'] = test_df[col].map(means).fillna(global_mean)
    
    return train_df, test_df

train_df, test_df = target_encode_country(train_df, test_df)
train_df = train_df.drop(columns=['country'])
test_df = test_df.drop(columns=['country'])

#age_group encoding
age_group_order = ['<18', '18-24', '25-44', '45-64', '65+']
age_group_map = {category: i for i, category in enumerate(age_group_order)}

# Fill missing with mode first
train_mode = train_df['age_group'].mode()[0]
train_df['age_group'] = train_df['age_group'].fillna(train_mode)
test_df['age_group'] = test_df['age_group'].fillna(train_mode)

train_df['age_group_encoded'] = train_df['age_group'].map(age_group_map)
test_df['age_group_encoded'] = test_df['age_group'].map(age_group_map)
train_df.drop('age_group', axis=1, inplace=True)
test_df.drop('age_group', axis=1, inplace=True)

#We drop the trip_id column as it is just an indentifier and does not have any correlation with the target variable

#Encoding days_booked_before trip
# These are ranges: "0-7", "8-14", "15-30", "31-60", "61-90", "90+"
booking_order = ['0-7', '8-14', '15-30', '31-60', '61-90', '90+']
booking_map = {category: i for i, category in enumerate(booking_order)}

# Fill missing with mode
train_mode = train_df['days_booked_before_trip'].mode()[0]
train_df['days_booked_before_trip'] = train_df['days_booked_before_trip'].fillna(train_mode)
test_df['days_booked_before_trip'] = test_df['days_booked_before_trip'].fillna(train_mode)

train_df['days_booked_encoded'] = train_df['days_booked_before_trip'].map(booking_map)
test_df['days_booked_encoded'] = test_df['days_booked_before_trip'].map(booking_map)

# Handle any unmapped values
train_df['days_booked_encoded'] = train_df['days_booked_encoded'].fillna(train_df['days_booked_encoded'].median())
test_df['days_booked_encoded'] = test_df['days_booked_encoded'].fillna(train_df['days_booked_encoded'].median())

train_df.drop('days_booked_before_trip', axis=1, inplace=True)
test_df.drop('days_booked_before_trip', axis=1, inplace=True)

#Encoding total_trip_days
# These are ranges: "1-3", "4-6", "7-14", "30+"
trip_days_order = ['1-3', '4-6', '7-14', '30+']
trip_days_map = {category: i for i, category in enumerate(trip_days_order)}

# Fill missing with mode
train_mode = train_df['total_trip_days'].mode()[0]
train_df['total_trip_days'] = train_df['total_trip_days'].fillna(train_mode)
test_df['total_trip_days'] = test_df['total_trip_days'].fillna(train_mode)

train_df['total_trip_days_encoded'] = train_df['total_trip_days'].map(trip_days_map)
test_df['total_trip_days_encoded'] = test_df['total_trip_days'].map(trip_days_map)

# Handle any unmapped values
train_df['total_trip_days_encoded'] = train_df['total_trip_days_encoded'].fillna(train_df['total_trip_days_encoded'].median())
test_df['total_trip_days_encoded'] = test_df['total_trip_days_encoded'].fillna(train_df['total_trip_days_encoded'].median())

train_df.drop('total_trip_days', axis=1, inplace=True)
test_df.drop('total_trip_days', axis=1, inplace=True)

train_df = train_df.drop(columns=['trip_id'])
test_trip_ids = test_df['trip_id'].copy()
test_df = test_df.drop(columns=['trip_id'])

# Get categorical columns (only object dtype)
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()
print(f"\nCategorical columns to one-hot encode: {categorical_cols}")

# Combine and encode
train_num_rows = train_df.shape[0]
combined_df = pd.concat([train_df, test_df], ignore_index=True)

combined_encoded_df = pd.get_dummies(combined_df, 
                                     columns=categorical_cols, 
                                     drop_first=True)

# Split back
train_df_encoded = combined_encoded_df.iloc[:train_num_rows].copy()
test_df_encoded = combined_encoded_df.iloc[train_num_rows:].copy()

print(f"Shape of encoded training data: {train_df_encoded.shape}")
print(f"Shape of encoded testing data: {test_df_encoded.shape}")


Categorical columns to one-hot encode: ['travel_companions', 'main_activity', 'visit_purpose', 'is_first_visit', 'tour_type', 'intl_transport_included', 'info_source', 'accomodation_included', 'food_included', 'domestic_transport_included', 'sightseeing_included', 'guide_included', 'insurance_included', 'arrival_weather', 'has_special_requirements', 'continent']
Shape of encoded training data: (12620, 67)
Shape of encoded testing data: (5852, 67)


In [7]:
X = train_df_encoded.drop('spend_category', axis=1)
y = train_df_encoded['spend_category']
X_test_final = test_df_encoded.drop('spend_category', axis=1, errors='ignore')

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Class distribution in y_train:\n{y_train.value_counts()}")


Shape of X_train: (10096, 66)
Shape of X_val: (2524, 66)
Class distribution in y_train:
spend_category
0.0    4996
1.0    3929
2.0    1171
Name: count, dtype: int64


In [8]:
numerical_cols = ['num_males', 'num_females', 'mainland_stay_nights', 
                  'island_stay_nights', 'total_travelers', 'female_ratio',
                  'total_stay_nights', 'island_ratio', 'country_freq', 
                  'country_target_enc', 'age_group_encoded', 'days_booked_encoded',
                  'total_trip_days_encoded', 'num_services_included', 'service_ratio']

scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val[numerical_cols] = scaler.transform(X_val[numerical_cols])
X_test_final[numerical_cols] = scaler.transform(X_test_final[numerical_cols])

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform
import numpy as np

print("\n" + "="*80)
print("HYPERPARAMETER TUNING: Random Forest with RandomizedSearchCV")
print("="*80)

# Define parameter distributions for random search
param_distributions = {
    'n_estimators': [200, 300, 400, 500],
    'max_depth': [10, 15, 20, 25, 30, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': ['balanced', 'balanced_subsample']
}

# Base model
rf_base = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)

# Stratified K-Fold to maintain class distribution
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_distributions,
    n_iter=50,  # Number of parameter settings sampled
    cv=cv,
    scoring='f1_weighted',  # Optimize for weighted F1
    n_jobs=-1,
    verbose=2,
    random_state=42,
    return_train_score=True
)

print("\nStarting RandomizedSearchCV... This may take a few minutes.")
random_search.fit(X_train, y_train)

print("\n" + "="*80)
print("BEST PARAMETERS FOUND:")
print("="*80)
for param, value in random_search.best_params_.items():
    print(f"{param}: {value}")

print(f"\nBest CV F1 Score (weighted): {random_search.best_score_:.4f}")

# Get the best model
best_rf_model = random_search.best_estimator_

# Evaluate on validation set
y_pred_tuned = best_rf_model.predict(X_val)

print("\n" + "="*80)
print("TUNED RANDOM FOREST - VALIDATION RESULTS:")
print("="*80)
print(classification_report(y_val, y_pred_tuned))
print(f"\nWeighted F1: {f1_score(y_val, y_pred_tuned, average='weighted'):.4f}")
print(f"Macro F1: {f1_score(y_val, y_pred_tuned, average='macro'):.4f}")
print(f"\nConfusion Matrix:\n{confusion_matrix(y_val, y_pred_tuned)}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n" + "="*80)
print("TOP 15 MOST IMPORTANT FEATURES:")
print("="*80)
print(feature_importance.head(15))


HYPERPARAMETER TUNING: Random Forest with RandomizedSearchCV

Starting RandomizedSearchCV... This may take a few minutes.
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END bootstrap=True, class_weight=balanced, max_depth=20, max_features=None, min_samples_leaf=6, min_samples_split=2, n_estimators=200; total time=  13.2s
[CV] END bootstrap=False, class_weight=balanced, max_depth=None, max_features=None, min_samples_leaf=8, min_samples_split=10, n_estimators=400; total time=  40.0s
[CV] END bootstrap=False, class_weight=balanced, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=400; total time=   8.6s
[CV] END bootstrap=False, class_weight=balanced, max_depth=None, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=400; total time=   8.2s
[CV] END bootstrap=False, class_weight=balanced, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=10, n_estimators=500; total time=   9.0s
[CV] E

In [9]:
# 1. Use your trained model to make predictions on the processed test data.
final_predictions = best_rf_model.predict(X_test_final)

# 2. Create a new DataFrame for the submission file.
#    The dictionary keys will become the column names.
submission_df = pd.DataFrame({
    'trip_id': test_trip_ids,
    'spend_category': final_predictions
})

# 3. Save the DataFrame to a CSV file.
#    index=False is essential to prevent writing an unwanted index column.
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' has been created successfully.")
print("\nFirst 5 rows of the submission file:")
print(submission_df.head())

Submission file 'submission.csv' has been created successfully.

First 5 rows of the submission file:
           trip_id  spend_category
0  tour_id8gzpck76             2.0
1  tour_idow1zxkou             0.0
2  tour_idue7esfqz             0.0
3  tour_idnj3mjzpb             0.0
4  tour_ida3us5yk2             0.0
